In [7]:

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import random
from pathlib import Path
from IPython.display import display, HTML

# %%
# PARAMETERS (edit these before running)
NUM_POINTS = 12           # number of delivery locations (excluding depot)
FIELD_SIZE = 100          # coordinate range (0..FIELD_SIZE)
NUM_NESTS = 40            # population of nests (solutions)
MAX_ITER = 200            # number of iterations
PA = 0.25                 # fraction of nests to abandon each generation
ALPHA = 0.01              # step size multiplier for Lévy flights (exploration scale)
BETA = 1.5                # Lévy exponent (1 < beta <= 2). Commonly 1.5 or 1.8
DRONE_MAX_LEG = 60.0      # maximum allowed leg distance before strong penalty
RANDOM_SEED = 42
SAVE_DIR = Path("/mnt/data")  # adjust or leave default for local Jupyter

np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# Print quick explanation of important parameters
print("EXPLANATION:")
print(f"- alpha (ALPHA) : step size multiplier that scales the Lévy flight steps. Larger alpha -> bigger jumps (more exploration). Here ALPHA = {ALPHA}.")
print(f"- beta (BETA) : Lévy exponent controlling the heavy-tailed distribution of step sizes. 1 < beta <= 2. Here BETA = {BETA}.")

# %%
# Problem setup: depot + delivery points
# You can either generate random points or load your own coordinates (see optional cell below)

def generate_random_problem(num_points=NUM_POINTS, field_size=FIELD_SIZE):
    depot = np.array([field_size/2, field_size/2])  # depot at center by default
    points = np.random.rand(num_points, 2) * field_size
    all_points = np.vstack([depot, points])  # index 0 is depot
    return all_points

all_points = generate_random_problem()
print(f"Generated {len(all_points)-1} delivery points + 1 depot. Depot at index 0.")

# Optional: display the points as a small table
points_df = pd.DataFrame(all_points, columns=["x","y"]) 
points_df.index.name = 'index'
display(points_df)


def route_from_key(key):
    """Random-key encoding: key is continuous vector of length NUM_POINTS; argsort gives permutation."""
    indices = np.argsort(key)
    route = [0] + (indices + 1).tolist() + [0]
    return route


def route_distance(route, points=all_points):
    coords = points[np.array(route)]
    dists = np.sqrt(((coords[1:] - coords[:-1])**2).sum(axis=1))
    return dists.sum(), dists


def objective_from_key(key, points=all_points, drone_max_leg=DRONE_MAX_LEG):
    route = route_from_key(key)
    total, dists = route_distance(route, points=points)
    penalty = 0.0
    over = dists - drone_max_leg
    if np.any(over > 0):
        penalty = 1e3 * (over[over>0]**2).sum()
    return total + penalty

# %%
# Lévy flight using Mantegna's algorithm

def levy_flight(beta, dim):
    """Return a vector of steps drawn from a Lévy distribution using Mantegna's algorithm."""
    # stability factor sigma_u
    sigma_u = (math.gamma(1+beta) * math.sin(math.pi*beta/2) / 
               (math.gamma((1+beta)/2) * beta * 2**((beta-1)/2))) ** (1/beta)
    u = np.random.normal(0, sigma_u, size=dim)
    v = np.random.normal(0, 1, size=dim)
    step = u / (np.abs(v) ** (1.0/beta))
    return step

# %%
# Cuckoo Search core loop

dim = all_points.shape[0] - 1  # number of deliveries
nests = np.random.rand(NUM_NESTS, dim)
fitness = np.array([objective_from_key(nests[i]) for i in range(NUM_NESTS)])
best_idx = np.argmin(fitness)
best_nest = nests[best_idx].copy()
best_fitness = fitness[best_idx]

history = []
print(f"Initial best fitness: {best_fitness:.6f}")
history.append({"iter": 0, "best_fitness": float(best_fitness), "mean_fitness": float(fitness.mean())})

for it in range(1, MAX_ITER+1):
    # Lévy flight updates
    for i in range(NUM_NESTS):
        current = nests[i].copy()
        step = levy_flight(BETA, dim)
        new_solution = current + ALPHA * step * (current - best_nest)
        new_solution = np.mod(new_solution, 1.0)
        new_fitness = objective_from_key(new_solution)
        if new_fitness < fitness[i]:
            nests[i] = new_solution
            fitness[i] = new_fitness

    # Abandon worst PA fraction
    K = int(PA * NUM_NESTS)
    if K > 0:
        worst_idx = np.argsort(fitness)[-K:]
        for idx in worst_idx:
            nests[idx] = np.random.rand(dim)
            fitness[idx] = objective_from_key(nests[idx])

    # Update global best
    cur_best_idx = np.argmin(fitness)
    cur_best_fitness = fitness[cur_best_idx]
    if cur_best_fitness < best_fitness:
        best_fitness = cur_best_fitness
        best_nest = nests[cur_best_idx].copy()

    history.append({"iter": it, "best_fitness": float(best_fitness), "mean_fitness": float(fitness.mean())})

    # Print iteration summary (every iteration)
    route = route_from_key(best_nest)
    if it % 1 == 0:
        print(f"Iter {it:3d} | Best fitness: {best_fitness:10.4f} | Mean fitness: {fitness.mean():10.4f}")

# %%
# Results, save history and plot
history_df = pd.DataFrame(history)
SAVE_DIR.mkdir(parents=True, exist_ok=True)
csv_path = SAVE_DIR / 'iteration_history.csv'
plot_path = SAVE_DIR / 'convergence.png'
history_df.to_csv(csv_path, index=False)

print("\nFINAL BEST ROUTE AND METRICS")
best_route = route_from_key(best_nest)
best_total, best_leg_distances = route_distance(best_route)
print(f"Best fitness (with penalties): {best_fitness:.6f}")
print(f"Best route (indices w.r.t all_points where 0 is depot): {best_route}")
print(f"Total route distance (no penalties): {best_total:.6f}")
print(f"Per-leg distances: {np.round(best_leg_distances,3).tolist()}")




EXPLANATION:
- alpha (ALPHA) : step size multiplier that scales the Lévy flight steps. Larger alpha -> bigger jumps (more exploration). Here ALPHA = 0.01.
- beta (BETA) : Lévy exponent controlling the heavy-tailed distribution of step sizes. 1 < beta <= 2. Here BETA = 1.5.
Generated 12 delivery points + 1 depot. Depot at index 0.


,x,y
index,,
0,50.000000,50.000000
1,37.454012,95.071431
2,73.199394,59.865848
3,15.601864,15.599452
4,5.808361,86.617615
5,60.111501,70.807258
6,2.058449,96.990985
7,83.244264,21.233911
8,18.182497,18.340451


Initial best fitness: 347963.970606
Iter   1 | Best fitness: 175404.1126 | Mean fitness: 1592798.3401
Iter   2 | Best fitness: 28670.1060 | Mean fitness: 1132628.6532
Iter   3 | Best fitness: 28670.1060 | Mean fitness: 1310212.0898
Iter   4 | Best fitness: 28670.1060 | Mean fitness: 977041.7139
Iter   5 | Best fitness: 28670.1060 | Mean fitness: 1094044.9271
Iter   6 | Best fitness: 28670.1060 | Mean fitness: 1204544.6585
Iter   7 | Best fitness: 28670.1060 | Mean fitness: 1286864.0984
Iter   8 | Best fitness: 28670.1060 | Mean fitness: 821965.2945
Iter   9 | Best fitness: 28670.1060 | Mean fitness: 945467.4046
Iter  10 | Best fitness: 28670.1060 | Mean fitness: 920619.0321
Iter  11 | Best fitness: 28670.1060 | Mean fitness: 745006.3869
Iter  12 | Best fitness: 28670.1060 | Mean fitness: 917378.1481
Iter  13 | Best fitness: 28670.1060 | Mean fitness: 615047.4237
Iter  14 | Best fitness: 28670.1060 | Mean fitness: 913245.2005
Iter  15 | Best fitness: 28670.1060 | Mean fitness: 822152.07